In [1]:
import torch as t

In [2]:
t.random.manual_seed(42)

In [3]:
from grid_world import GridWorld
from grid_world import STATES, ACTIONS
from agent import Agent, run_episode

In [4]:
n_states, n_actions = len(STATES), len(ACTIONS)

In [5]:
N = 20
TRIALS = 300

In [6]:
agent = Agent(n_states, n_actions, sigma=0.9)
G = 0
for i in range(N):
    G += run_episode(agent, GridWorld())

returns = [G]

for trial in range(TRIALS):
    G_ = 0
    theta = agent.theta
    theta_ = agent.sampleTheta()
    agent.theta = theta_
    for i in range(N):
        G_ += run_episode(agent, GridWorld())
    agent.theta = theta
    returns.append(G_)

    if G_ > G:
        agent.theta = theta_
        G = G_
        print(f"trial={trial}, avg return={G_/N}")

trial=0, avg return=-87.5
trial=4, avg return=-41.5
trial=5, avg return=-30.0
trial=6, avg return=-22.5
trial=9, avg return=-11.5
trial=10, avg return=7.5
trial=15, avg return=8.5
trial=20, avg return=9.0
trial=26, avg return=10.0
